In [180]:
from gurobipy import Model, GRB, quicksum
import math

In [181]:
model = Model("Courier_Assignment_MultiObjective")

In [182]:
# Periods
T = [1]  # Periods

# Orders per period
O_t = {1: [1, 2, 3]}

# Couriers per period
C_t = {1: [1, 2]}

# Add parameter `a`
a = {
    (i, j, t): (
        1 if (j % 2 == 0) else 0
    )  # even couriers always accept, odd never accept
    for t in T
    for i in O_t[t]
    for j in C_t[t]
}

# Preparation times for orders
prep_time = {1: 10, 2: 15, 3: 20}

# Arrival times for couriers (i,j,t):time
courier_arrival_time = {
    (1, 1, 1): 20,
    (2, 1, 1): 12,
    (3, 1, 1): 18,
    (1, 2, 1): 10,
    (2, 2, 1): 14,
    (3, 2, 1): 8,
}

# Locations of orders
order_location = {
    1: (0, 0),
    2: (1, 1),
    3: (2, 2),
}

# Start locations of couriers
courier_start_location = {
    1: (5, 4),
    2: (1, 0),
}

In [183]:
# Distance function
def distance(i, j):
    ox, oy = order_location[i]
    cx, cy = courier_start_location[j]
    return math.sqrt((ox - cx) ** 2 + (oy - cy) ** 2)

In [184]:
# Decision variables
x = model.addVars(
    [(i, j, t) for t in T for i in O_t[t] for j in C_t[t]],
    vtype=GRB.BINARY,
    name="x",
)

In [185]:
# Workload
workload = {
    j: quicksum(x[i, j, t] for t in T for i in O_t[t] if (i, j, t) in x)
    for j in set(j for t in T for j in C_t[t])
}

# Mean workload
mean_workload = quicksum(workload[j] for j in workload) / len(workload)

# Auxiliary variables for workload imbalance
y = model.addVars(workload.keys(), lb=0, name="y")

In [186]:
# Objectives
waiting_time_objective = quicksum(
    abs(prep_time[i] - courier_arrival_time[i, j, t]) * x[i, j, t]
    for t in T
    for i in O_t[t]
    for j in C_t[t]
)
distance_objective = quicksum(
    distance(i, j) * x[i, j, t] for t in T for i in O_t[t] for j in C_t[t]
)
workload_imbalance_objective = quicksum(y[j] for j in workload)

# Add objectives
model.setObjectiveN(
    waiting_time_objective, index=0, priority=1, name="MinimizeWaitingTime"
)
model.setObjectiveN(distance_objective, index=1, priority=1, name="MinimizeDistance")
model.setObjectiveN(
    workload_imbalance_objective, index=2, priority=1, name="MinimizeWorkloadImbalance"
)

In [187]:
# Constraints
for t in T:
    for i in O_t[t]:
        model.addConstr(
            quicksum(x[i, j, t] for j in C_t[t]) == 1, name=f"assign_{i}_{t}"
        )

for t in T:
    for i in O_t[t]:
        for j in C_t[t]:
            model.addConstr(x[i, j, t] <= a[i, j, t], name=f"acceptance_{i}_{j}_{t}")

for j in workload:
    model.addConstr(y[j] >= workload[j] - mean_workload, name=f"abs_dev_pos_{j}")
    model.addConstr(y[j] >= mean_workload - workload[j], name=f"abs_dev_neg_{j}")

In [188]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 8 columns and 40 nonzeros
Model fingerprint: 0x423654dd
Variable types: 2 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined)...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 13 rows, 8 columns and 40 nonzeros
Model fingerprint: 0xefb

In [189]:
print("\nObjective Values:")
print(f"Total Waiting Time: {waiting_time_objective.getValue()}")
print(f"Total Distance Traveled: {distance_objective.getValue()}")
print(f"Workload Imbalance: {workload_imbalance_objective.getValue()}")

# Print assignment results
print("\nAssignments:")
for t in T:
    for i in O_t[t]:
        for j in C_t[t]:
            if x[i, j, t].x == 1:  # Check if variable is assigned
                print(f"Order {i} is assigned to Courier {j} in Time Period {t}")


Objective Values:
Total Waiting Time: 13.0
Total Distance Traveled: 4.23606797749979
Workload Imbalance: 3.0

Assignments:
Order 1 is assigned to Courier 2 in Time Period 1
Order 2 is assigned to Courier 2 in Time Period 1
Order 3 is assigned to Courier 2 in Time Period 1
